# UKy Flowsheet Tutorial 

This tutorial will show how to build, initialize, and simulate the West Kentucky No.13 Coal Refuse flowsheet. 

- Useful Links:
    - Public Github Repository: https://github.com/prommis/prommis/tree/main
    - UKy Flowsheet Code: https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py

![uky_flowsheet.png](./uky_flowsheet.png)


## Step 1: Import the necessary tools

In [1]:
# Import the essentials from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Constraint,
    SolverFactory,
    Suffix,
    TransformationFactory,
    Var,
    value,
    units,
)
from pyomo.network import Arc, SequentialDecomposition
from pyomo.util.check_units import assert_units_consistent

# Import the essentials from IDAES
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock,
)
from idaes.models_extra.power_generation.properties.natural_gas_PR import (
    EosType,
    get_prop,
)
from idaes.core import (
    FlowDirection,
    FlowsheetBlock,
    MaterialBalanceType,
    MomentumBalanceType,
)

# Import initializtion and diagnostic tools from IDAES
from idaes.core.initialization import BlockTriangularizationInitializer
from idaes.core.util.initialization import propagate_state
from idaes.core.util.model_diagnostics import DiagnosticsToolbox
from idaes.core.util.model_statistics import degrees_of_freedom

# Import unit models from IDAES
from idaes.models.unit_models.feed import Feed, FeedInitializer
from idaes.models.unit_models.mixer import Mixer, MixingType, MomentumMixingType
from idaes.models.unit_models.mscontactor import MSContactor, MSContactorInitializer
from idaes.models.unit_models.product import Product, ProductInitializer
from idaes.models.unit_models.separator import (
    EnergySplittingType,
    Separator,
    SplittingType,
)
from idaes.models.unit_models.solid_liquid import SLSeparator

# Import the UKy-specific unit and property models
from prommis.leaching.leach_reactions import CoalRefuseLeachingReactions
from prommis.leaching.leach_solids_properties import CoalRefuseParameters
from prommis.leaching.leach_solution_properties import LeachSolutionParameters
from prommis.precipitate.precipitate_liquid_properties import AqueousParameter
from prommis.precipitate.precipitate_solids_properties import PrecipitateParameters
from prommis.precipitate.precipitator import Precipitator
from prommis.roasting.ree_oxalate_roaster import REEOxalateRoaster
from prommis.solvent_extraction.ree_aq_distribution import REESolExAqParameters
from prommis.solvent_extraction.ree_og_distribution import REESolExOgParameters
from prommis.solvent_extraction.solvent_extraction import SolventExtraction

## Step 2 Flowsheet building

### Step 2.1: Create Flowsheet
Start by creating a pyomo model and a flowsheet.

In [2]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

Then begin assembling the unit, property, and reaction models section-by-section. These variables will be created in chronological order - beginning with the leaching section of the flowsheet and concluding with the product roasting section.

### Step 2.2 Create variables for the leaching section
Specify the necessary unit, property, and reaction models for the leaching section.

In [3]:
# Leaching property models
m.fs.leach_soln = LeachSolutionParameters()
m.fs.coal = CoalRefuseParameters()

# Leaching reaction model
m.fs.leach_rxns = CoalRefuseLeachingReactions()

# Leaching unit model
m.fs.leach = MSContactor(
    number_of_finite_elements=2,
    streams={
        "liquid": {
            "property_package": m.fs.leach_soln,
            "has_energy_balance": False,
            "has_pressure_balance": False,
        },
        "solid": {
            "property_package": m.fs.coal,
            "has_energy_balance": False,
            "has_pressure_balance": False,
        },
    },
    heterogeneous_reactions=m.fs.leach_rxns,
)

# Define leach tank volume
m.fs.leach.volume = Var(
    m.fs.time,
    m.fs.leach.elements,
    initialize=1,
    units=units.litre,
    doc="Volume of each finite element.",
)


# Define constraint for heterogeneous reaction extent
def rule_heterogeneous_reaction_extent(b, t, s, r):
    return (
        b.heterogeneous_reaction_extent[t, s, r]
        == b.heterogeneous_reactions[t, s].reaction_rate[r] * b.volume[t, s]
    )


m.fs.leach.heterogeneous_reaction_extent_constraint = Constraint(
    m.fs.time,
    m.fs.leach.elements,
    m.fs.leach_rxns.reaction_idx,
    rule=rule_heterogeneous_reaction_extent,
)

# Solid-liquid separator used to approximate a filter press
m.fs.sl_sep1 = SLSeparator(
    solid_property_package=m.fs.coal,
    liquid_property_package=m.fs.leach_soln,
    material_balance_type=MaterialBalanceType.componentTotal,
    # Ignore momentum balance since the property package does not have pressure or momentum terms
    momentum_balance_type=MomentumBalanceType.none,
    # Ignore energy split basis since the property package does not have temperature terms
    energy_split_basis=EnergySplittingType.none,
)

# Recycle loop mixer
m.fs.leach_mixer = Mixer(
    property_package=m.fs.leach_soln,
    num_inlets=2,
    inlet_list=["recycle", "feed"],
    material_balance_type=MaterialBalanceType.componentTotal,
    # Ignore mixing type since the property package does not have enthalpy terms
    energy_mixing_type=MixingType.none,
    # Ignore momentum mixing since the property package does not have pressure or momentum terms
    momentum_mixing_type=MomentumMixingType.none,
)

# Define inlets into the flowsheet
m.fs.leach_liquid_feed = Feed(property_package=m.fs.leach_soln)
m.fs.leach_solid_feed = Feed(property_package=m.fs.coal)

# Define outlets from the flowsheet
m.fs.leach_filter_cake = Product(property_package=m.fs.coal)
m.fs.leach_filter_cake_liquid = Product(property_package=m.fs.leach_soln)

### Step 2.3 Create variables for the solvent extraction section
Specify the necessary unit, property, and reaction models for the solvent extraction section.

In [4]:
# Solvent extraction property models
m.fs.prop_a = REESolExAqParameters()
m.fs.prop_o = REESolExOgParameters()

# First unit model in the solvent extraction circuit (cleaner defined in the precipitation section)
m.fs.solex_rougher = SolventExtraction(
    number_of_finite_elements=3,
    dynamic=False,
    aqueous_stream={
        "property_package": m.fs.leach_soln,
        "flow_direction": FlowDirection.forward,
        "has_energy_balance": False,
        "has_pressure_balance": False,
    },
    organic_stream={
        "property_package": m.fs.prop_o,
        "flow_direction": FlowDirection.backward,
        "has_energy_balance": False,
        "has_pressure_balance": False,
    },
)

# Recycle loop separator
m.fs.sep1 = Separator(
    property_package=m.fs.leach_soln,
    outlet_list=["recycle", "purge"],
    split_basis=SplittingType.totalFlow,
    material_balance_type=MaterialBalanceType.componentTotal,
    momentum_balance_type=MomentumBalanceType.none,
    energy_split_basis=EnergySplittingType.none,
)

# Mixes the solvent extraction outlet streams prior to feeding to the precipitator
m.fs.sx_mixer = Mixer(
    property_package=m.fs.prop_o,
    num_inlets=2,
    inlet_list=["aqueous_inlet", "organic_inlet"],
    material_balance_type=MaterialBalanceType.componentTotal,
    energy_mixing_type=MixingType.none,
    momentum_mixing_type=MomentumMixingType.none,
)

# Define outlets from the flowsheet
m.fs.recycle1_purge = Product(property_package=m.fs.leach_soln)

m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Al"] = 5.2 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Ca"] = 3.0 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Fe"] = 24.7 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Sc"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Y"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "La"] = 32.4 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Ce"] = 58.2 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Pr"] = 58.2 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Nd"] = 87.6 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Sm"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Gd"] = 69.8 / 100
m.fs.solex_rougher.partition_coefficient[1, "aqueous", "organic", "Dy"] = 96.6 / 100

m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Al"] = 4.9 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Ca"] = 12.3 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Fe"] = 6.4 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Sc"] = 16.7 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Y"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "La"] = 23.2 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Ce"] = 24.9 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Pr"] = 15.1 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Nd"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Sm"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Gd"] = 7.6 / 100
m.fs.solex_rougher.partition_coefficient[2, "aqueous", "organic", "Dy"] = 5.0 / 100

m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Al"] = 4.9 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Ca"] = 12.3 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Fe"] = 6.4 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Sc"] = 16.7 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Y"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "La"] = 23.2 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Ce"] = 24.9 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Pr"] = 15.1 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Nd"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Sm"] = 99.9 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Gd"] = 7.6 / 100
m.fs.solex_rougher.partition_coefficient[3, "aqueous", "organic", "Dy"] = 5.0 / 100

### Step 2.3 Create variables for the precipitation section
Specify the necessary unit, property, and reaction models for the precipitation section.

In [5]:
# Need to define the set of key ions
key_components = {
    "H^+",
    "Ce^3+",
    "Al^3+",
    "Fe^3+",
    "Ca^2+",
    "C2O4^2-",
}

# Precipitation property packages
m.fs.properties_aq = AqueousParameter()
m.fs.properties_solid = PrecipitateParameters()

# Second unit model in the solvent extraction circuit - requires the aqueous precipitation package
m.fs.solex_cleaner = SolventExtraction(
    number_of_finite_elements=3,
    dynamic=False,
    aqueous_stream={
        "property_package": m.fs.properties_aq,
        "flow_direction": FlowDirection.forward,
        "has_energy_balance": False,
        "has_pressure_balance": False,
    },
    organic_stream={
        "property_package": m.fs.prop_o,
        "flow_direction": FlowDirection.backward,
        "has_energy_balance": False,
        "has_pressure_balance": False,
    },
)

# Precipitation unit model
m.fs.precipitator = Precipitator(
    property_package_aqueous=m.fs.properties_aq,
    property_package_precipitate=m.fs.properties_solid,
)

# Solid-liquid separator used to approximate a filter press
m.fs.sl_sep2 = SLSeparator(
    solid_property_package=m.fs.properties_solid,
    liquid_property_package=m.fs.properties_aq,
    material_balance_type=MaterialBalanceType.componentTotal,
    momentum_balance_type=MomentumBalanceType.none,
    energy_split_basis=EnergySplittingType.none,
)

# Recycle loop separator
m.fs.sep2 = Separator(
    property_package=m.fs.properties_aq,
    outlet_list=["recycle", "purge"],
    split_basis=SplittingType.totalFlow,
    material_balance_type=MaterialBalanceType.componentTotal,
    momentum_balance_type=MomentumBalanceType.none,
    energy_split_basis=EnergySplittingType.none,
)

# Define outlets from the flowsheet
m.fs.recycle2_purge = Product(property_package=m.fs.properties_aq)

m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Al"] = 3.6 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Ca"] = 3.7 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Fe"] = 2.1 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Sc"] = 99.9 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Y"] = 99.9 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "La"] = 75.2 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Ce"] = 95.7 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Pr"] = 96.5 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Nd"] = 99.2 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Sm"] = 99.9 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Gd"] = 98.6 / 100
m.fs.solex_cleaner.partition_coefficient[:, "aqueous", "organic", "Dy"] = 99.9 / 100

### Step 2.3 Create variables for the product roaster section
Specify the necessary unit, property, and reaction models for the roaster section.

In [6]:
# Define the relevant gas species
gas_species = {"O2", "H2O", "CO2", "N2"}

# Roaster property packages
m.fs.prop_gas = GenericParameterBlock(
    **get_prop(gas_species, ["Vap"], EosType.IDEAL),
    doc="gas property",
)
m.fs.prop_solid = PrecipitateParameters(
    key_components=key_components,
)

# Roaster unit model
m.fs.roaster = REEOxalateRoaster(
    property_package_gas=m.fs.prop_gas,
    property_package_precipitate=m.fs.prop_solid,
    has_holdup=False,
    has_heat_transfer=True,
    has_pressure_change=True,
)

### Step 2.4 Connect the unit models
Next, use Pyomo arcs as streams to connect the units as portrayed in the flowsheet image above.

In [7]:
m.fs.sol_feed = Arc(
    source=m.fs.leach_solid_feed.outlet, destination=m.fs.leach.solid_inlet
)
m.fs.liq_feed = Arc(
    source=m.fs.leach_liquid_feed.outlet, destination=m.fs.leach_mixer.feed
)
m.fs.feed_mixture = Arc(
    source=m.fs.leach_mixer.outlet, destination=m.fs.leach.liquid_inlet
)
m.fs.s01 = Arc(source=m.fs.leach.solid_outlet, destination=m.fs.sl_sep1.solid_inlet)
m.fs.s02 = Arc(source=m.fs.leach.liquid_outlet, destination=m.fs.sl_sep1.liquid_inlet)
m.fs.sep1_solid = Arc(
    source=m.fs.sl_sep1.solid_outlet, destination=m.fs.leach_filter_cake.inlet
)
m.fs.sep1_retained_liquid = Arc(
    source=m.fs.sl_sep1.retained_liquid_outlet,
    destination=m.fs.leach_filter_cake_liquid.inlet,
)
m.fs.sep1_liquid = Arc(
    source=m.fs.sl_sep1.recovered_liquid_outlet,
    destination=m.fs.solex_rougher.mscontactor.aqueous_inlet,
)
m.fs.recycle1 = Arc(
    source=m.fs.solex_rougher.mscontactor.aqueous_outlet,
    destination=m.fs.sep1.inlet,
)
m.fs.purge1 = Arc(source=m.fs.sep1.purge, destination=m.fs.recycle1_purge.inlet)
m.fs.recycle_feed = Arc(source=m.fs.sep1.recycle, destination=m.fs.leach_mixer.recycle)
m.fs.s03 = Arc(
    source=m.fs.solex_rougher.mscontactor.organic_outlet,
    destination=m.fs.solex_cleaner.mscontactor.organic_inlet,
)
m.fs.s04 = Arc(
    source=m.fs.solex_cleaner.mscontactor.aqueous_outlet,
    destination=m.fs.sx_mixer.aqueous_inlet,
)
m.fs.s05 = Arc(
    source=m.fs.solex_cleaner.mscontactor.organic_outlet,
    destination=m.fs.sx_mixer.organic_inlet,
)
m.fs.s06 = Arc(
    source=m.fs.sx_mixer.outlet,
    destination=m.fs.precipitator.aqueous_inlet,
)
m.fs.s07 = Arc(
    source=m.fs.precipitator.precipitate_outlet,
    destination=m.fs.sl_sep2.solid_inlet,
)
m.fs.s08 = Arc(
    source=m.fs.precipitator.aqueous_outlet, destination=m.fs.sl_sep2.liquid_inlet
)
m.fs.sep2_solid = Arc(
    source=m.fs.sl_sep2.solid_outlet, destination=m.fs.roaster.solid_inlet
)

m.fs.sep2_recovered_liquid = Arc(
    source=m.fs.sl_sep2.recovered_liquid_outlet, destination=m.fs.sep2.inlet
)
m.fs.purge2 = Arc(source=m.fs.sep2.purge, destination=m.fs.recycle2_purge.inlet)
m.fs.recycle2 = Arc(
    source=m.fs.sep2.recycle,
    destination=m.fs.solex_cleaner.mscontactor.aqueous_inlet,
)

TransformationFactory("network.expand_arcs").apply_to(m)

### Step 2.5 Apply scaling
In order for the flowsheet to solve, variables will need to be scaled appropriately. While variables often have a default scaling set, it is important to re-scale those with poor initial scaling in this flowsheet as there are many trace components resulting in variables with very small magnitudes.

In [8]:
def set_scaling(m):
    m.scaling_factor = Suffix(direction=Suffix.EXPORT)

    # Create a sets to target variables with these specific components
    component_set1 = [
        "H2O",
        "H",
        "HSO4",
        "SO4",
        "Sc",
        "Y",
        "La",
        "Ce",
        "Pr",
        "Nd",
        "Sm",
        "Gd",
        "Dy",
        "Al",
        "Ca",
        "Fe",
    ]

    component_set2 = [
        "Sc",
        "Y",
        "La",
        "Ce",
        "Pr",
        "Nd",
        "Sm",
        "Gd",
        "Dy",
        "Al",
        "Ca",
        "Fe",
    ]

    # Apply scaling to badly scaled variables using the first component set
    for component in component_set1:
        m.scaling_factor[m.fs.leach.liquid[0, 1].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.leach.liquid_inlet_state[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[
            m.fs.leach_liquid_feed.properties[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.sl_sep1.liquid_inlet_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.sl_sep1.split.recovered_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.sl_sep1.split.retained_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.leach_filter_cake_liquid.properties[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.aqueous[0, 1].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.aqueous[0, 2].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.aqueous[0, 3].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.aqueous_inlet_state[0].conc_mol_comp[
                component
            ]
        ] = 1e5
        m.scaling_factor[m.fs.sep1.mixed_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.sep1.recycle_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.sep1.purge_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.recycle1_purge.properties[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[m.fs.leach_mixer.recycle_state[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[m.fs.leach_mixer.feed_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.leach_mixer.mixed_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.leach.liquid_inlet_state[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[m.fs.leach.liquid_inlet_state[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[m.fs.leach.liquid_inlet_state[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[m.fs.leach.liquid_inlet_state[0].conc_mol_comp[component]] = (
            1e5
        )

    # Apply scaling to badly scaled variables using the second component set
    for component in component_set2:
        m.scaling_factor[
            m.fs.sl_sep2.liquid_inlet_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.sl_sep2.split.recovered_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.sl_sep2.split.retained_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.organic[0, 1].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.organic[0, 2].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.organic[0, 3].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mol_comp[
                component
            ]
        ] = 1e5
        m.scaling_factor[m.fs.sep2.mixed_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.sep2.recycle_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.sep2.purge_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[m.fs.recycle2_purge.properties[0].conc_mol_comp[component]] = (
            1e5
        )
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.aqueous[0, 1].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.aqueous[0, 2].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.aqueous[0, 3].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.aqueous_inlet_state[0].conc_mol_comp[
                component
            ]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.organic[0, 1].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.organic[0, 2].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.organic[0, 3].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.solex_cleaner.mscontactor.organic_inlet_state[0].conc_mol_comp[
                component
            ]
        ] = 1e5
        m.scaling_factor[
            m.fs.sx_mixer.aqueous_inlet_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.sx_mixer.organic_inlet_state[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[m.fs.sx_mixer.mixed_state[0].conc_mol_comp[component]] = 1e5
        m.scaling_factor[
            m.fs.precipitator.cv_aqueous.properties_in[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[
            m.fs.precipitator.cv_aqueous.properties_out[0].conc_mol_comp[component]
        ] = 1e5
        m.scaling_factor[m.fs.leach.liquid_inlet_state[0].conc_mol_comp[component]] = (
            1e5
        )

    # Apply scaling to the remaining badly scaled variables
    m.scaling_factor[m.fs.sl_sep2.solid_state[0].temperature] = 1e-2
    m.scaling_factor[m.fs.sl_sep2.liquid_inlet_state[0].flow_vol] = 1e-2
    m.scaling_factor[m.fs.sl_sep2.split.recovered_state[0].flow_vol] = 1e-2
    m.scaling_factor[m.fs.sl_sep2.split.retained_state[0].flow_vol] = 1e-2

    m.scaling_factor[m.fs.sep2.mixed_state[0].flow_vol] = 1e-2
    m.scaling_factor[m.fs.sep2.recycle_state[0].flow_vol] = 1e-2
    m.scaling_factor[m.fs.sep2.purge_state[0].flow_vol] = 1e-2
    m.scaling_factor[m.fs.recycle2_purge.properties[0].flow_vol] = 1e-2

    m.scaling_factor[m.fs.solex_cleaner.mscontactor.aqueous[0, 1].flow_vol] = 1e-2
    m.scaling_factor[m.fs.solex_cleaner.mscontactor.aqueous[0, 2].flow_vol] = 1e-2
    m.scaling_factor[m.fs.solex_cleaner.mscontactor.aqueous[0, 3].flow_vol] = 1e-2
    m.scaling_factor[m.fs.solex_cleaner.mscontactor.aqueous_inlet_state[0].flow_vol] = (
        1e-2
    )
    m.scaling_factor[m.fs.solex_cleaner.mscontactor.organic[0, 1].flow_vol] = 1e-2
    m.scaling_factor[m.fs.solex_cleaner.mscontactor.organic[0, 2].flow_vol] = 1e-2
    m.scaling_factor[m.fs.solex_cleaner.mscontactor.organic[0, 3].flow_vol] = 1e-2

    m.scaling_factor[m.fs.precipitator.cv_precipitate[0].temperature] = 1e2

    m.scaling_factor[m.fs.precipitator.cv_aqueous.properties_in[0].flow_vol] = 1e-2
    m.scaling_factor[m.fs.precipitator.cv_aqueous.properties_out[0].flow_vol] = 1e-2

    m.scaling_factor[m.fs.roaster.gas_in[0].flow_mol] = 1e-3
    m.scaling_factor[m.fs.roaster.gas_in[0].flow_mol_phase["Vap"]] = 1e-3
    m.scaling_factor[m.fs.roaster.gas_in[0].temperature] = 1e-2
    m.scaling_factor[m.fs.roaster.gas_in[0].pressure] = 1e-5
    m.scaling_factor[m.fs.roaster.gas_out[0].flow_mol_phase["Vap"]] = 1e-3
    m.scaling_factor[m.fs.roaster.gas_out[0].flow_mol] = 1e-3
    m.scaling_factor[m.fs.roaster.gas_out[0].temperature] = 1e-2
    m.scaling_factor[m.fs.roaster.gas_out[0].pressure] = 1e-5
    m.scaling_factor[m.fs.roaster.solid_in[0].temperature] = 1e-2

    # Reassign m to use the new scaling factors
    scaling = TransformationFactory("core.scale_model")
    scaled_model = scaling.create_using(m, rename=False)

    return scaled_model

### Step 2.6 Set the operating conditions

Now specify the inlet feed conditions into the flowsheet and fix any necessary unit model variables such that the
degrees of freedom of the system are equal to zero.

In [9]:
eps = 1e-7 * units.mg / units.L

# Fix the volume of the leaching tank
m.fs.leach.volume.fix(100 * units.gallon)

# Specify the flow rate and mass concentration of the liquid feed to the leaching unit
m.fs.leach_liquid_feed.flow_vol.fix(224.3 * units.L / units.hour)
m.fs.leach_liquid_feed.conc_mass_comp.fix(1e-10 * units.mg / units.L)
m.fs.leach_liquid_feed.conc_mass_comp[0, "H"].fix(2 * 0.05 * 1e3 * units.mg / units.L)
m.fs.leach_liquid_feed.conc_mass_comp[0, "HSO4"].fix(1e-8 * units.mg / units.L)
m.fs.leach_liquid_feed.conc_mass_comp[0, "SO4"].fix(0.05 * 96e3 * units.mg / units.L)

# Specify the mass flow and mass fractions of the solid feed to the leaching unit
m.fs.leach_solid_feed.flow_mass.fix(22.68 * units.kg / units.hour)
m.fs.leach_solid_feed.mass_frac_comp[0, "inerts"].fix(0.6952 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Al2O3"].fix(0.237 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Fe2O3"].fix(0.0642 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "CaO"].fix(3.31e-3 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Sc2O3"].fix(2.77966e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Y2O3"].fix(3.28653e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "La2O3"].fix(6.77769e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Ce2O3"].fix(0.000156161 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Pr2O3"].fix(1.71438e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Nd2O3"].fix(6.76618e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Sm2O3"].fix(1.47926e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Gd2O3"].fix(1.0405e-05 * units.kg / units.kg)
m.fs.leach_solid_feed.mass_frac_comp[0, "Dy2O3"].fix(7.54827e-06 * units.kg / units.kg)

# Fix the flow rate and mass concentration of the organic inlet to the solvent extraction rougher
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].flow_vol.fix(
    62.01 * units.L / units.hour
)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Al"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Ca"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Fe"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Sc"].fix(
    321.34 * units.mg / units.L
)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Y"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["La"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Ce"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Pr"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Nd"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Sm"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Gd"].fix(eps)
m.fs.solex_rougher.mscontactor.organic_inlet_state[0].conc_mass_comp["Dy"].fix(eps)

# Fix the amount of liquid recovered from the two solid-liquid separators
m.fs.sl_sep1.liquid_recovery.fix(
    0.7
)  # 70% of liquid is recovered and 30% is retained in the filter cake
m.fs.sl_sep2.liquid_recovery.fix(0.7)

# Fix how much material is recycled in the two recycle loops
m.fs.sep1.split_fraction[:, "recycle"].fix(
    0.9
)  # 90% of the stream is recovered and 10% is purged
m.fs.sep2.split_fraction[:, "recycle"].fix(0.9)

# Fix the inlet temperature of the precipitator's precipitate inlet
m.fs.precipitator.cv_precipitate[0].temperature.fix(348.15 * units.K)
m.fs.precipitator.cv_precipitate[0].temperature.setlb(0)

# Fix the operating conditions of the roaster
m.fs.roaster.deltaP.fix(0)
m.fs.roaster.gas_inlet.temperature.fix(1330)
m.fs.roaster.gas_inlet.pressure.fix(101325)
m.fs.roaster.solid_in[0].temperature.setlb(0)
m.fs.roaster.gas_out[0].temperature.setub(2000)
m.fs.roaster.gas_in[0].temperature.setub(2000)
m.fs.roaster.gas_in[0].pressure.setub(1e6)

fgas = 0.00781  # Inlet flue gas mole flow rate

# Fix the inlet flue gas composition - typical flue gas by burning CH4 with air has a stoichiometric ratio of 2.3
gas_comp = {
    "O2": 0.1118,
    "H2O": 0.1005,
    "CO2": 0.0431,
    "N2": 0.7446,
}
for i, v in gas_comp.items():
    m.fs.roaster.gas_inlet.mole_frac_comp[0, i].fix(v)
m.fs.roaster.gas_inlet.flow_mol.fix(fgas)

# Fix the outlet product temperature
m.fs.roaster.gas_outlet.temperature.fix(873.15)

# Fix the operating conditions
m.fs.roaster.flow_mol_moist_feed.fix(6.75e-4)
m.fs.roaster.frac_comp_recovery.fix(0.95)

# Step 3: Solve the square problem
## Step 3.1: Initialize the system
Since there are two recycle loops involved, sequential decomposition will be used to initialize the flowsheet and the two tear sets must be specified to successfully initialize the system.

In [10]:
def initialize_system(m):
    # Initialize the model with sequential decomposition
    seq = SequentialDecomposition()
    seq.options.tear_method = "Direct"
    seq.options.iterLim = 1
    seq.options.tear_set = [m.fs.feed_mixture, m.fs.recycle2]

    # Print initialization order for user visibility
    G = seq.create_graph(m)
    order = seq.calculation_order(G)
    print("Initialization Order")
    for o in order:
        print(o[0].name)

    # Create tear guesses with initial guesses that are close to the solution
    tear_guesses1 = {
        "flow_mass": {0.007},
        "conc_mass_comp": {
            ("Al"): 1493939.39,
            ("Ca"): 501864.01,
            ("Ce"): 49698.79,
            ("Dy"): 466.86,
            ("Fe"): 1685228.86,
            ("Gd"): 1624.81,
            ("La"): 32143.22,
            ("Nd"): 12552.13,
            ("Pr"): 4084.40,
            ("Sc"): 17310.17,
            ("Sm"): 931.35,
            ("Y"): 2666.95,
        },
        "flow_mol_comp": {
            ("Al"): 577.62,
            ("Ca"): 64.65,
            ("Ce"): 1.17,
            ("Dy"): 0.0086,
            ("Fe"): 231.93,
            ("Gd"): 0.054,
            ("La"): 0.63,
            ("Nd"): 0.34,
            ("Pr"): 0.099,
            ("Sc"): 303.33,
            ("Sm"): 0.021,
            ("Y"): 0.090,
        },
    }
    tear_guesses2 = {
        "flow_mol_comp": {
            ("Al2(C2O4)3(s)"): 1.76,
            ("Ce2(C2O4)3(s)"): 2.65,
            ("Dy2(C2O4)3(s)"): 0.068,
            ("Fe2(C2O4)3(s)"): 2.64,
            ("Gd2(C2O4)3(s)"): 0.27,
            ("La2(C2O4)3(s)"): 0.86,
            ("Nd2(C2O4)3(s)"): 1.35,
            ("Pr2(C2O4)3(s)"): 0.36,
            ("Sc2(C2O4)3(s)"): 0.62,
            ("Sm2(C2O4)3(s)"): 0.15,
            ("Y2(C2O4)3(s)"): 0.31,
        },
    }
    tear_guesses3 = {
        "flow_vol": {0: 747.99},
        "conc_mass_comp": {
            (0, "Al"): 180.84,
            (0, "Ca"): 28.93,
            (0, "Ce"): 5.48,
            (0, "Dy"): 4.46e-11,
            (0, "Fe"): 269.98,
            (0, "Gd"): 2.60e-7,
            (0, "H"): 20.06,
            (0, "H2O"): 1000000,
            (0, "HSO4"): 963.06,
            (0, "La"): 0.0037,
            (0, "Nd"): 1.81e-7,
            (0, "Pr"): 3.65e-6,
            (0, "SO4"): 486.24,
            (0, "Sc"): 4.17e-11,
            (0, "Sm"): 6.30e-10,
            (0, "Y"): 7.18e-11,
        },
    }
    tear_guesses4 = {
        "flow_vol": {0: 157.14},
        "conc_mass_comp": {
            (0, "Al"): 380.67,
            (0, "Ca"): 122.97,
            (0, "Ce"): 5.67,
            (0, "Dy"): 0.15,
            (0, "Fe"): 741.24,
            (0, "Gd"): 0.56,
            (0, "H"): 2.274,
            (0, "H2O"): 1000000,
            (0, "HSO4"): 881.19,
            (0, "La"): 2.24,
            (0, "Nd"): 2.68,
            (0, "Pr"): 0.72,
            (0, "SO4"): 3924.07,
            (0, "Sc"): 0.10,
            (0, "Sm"): 0.30,
            (0, "Y"): 0.39,
        },
    }

    # Pass the tear guesses to the sequential decomposition tool
    seq.set_guesses_for(m.fs.precipitator.cv_aqueous.properties_out[0], tear_guesses1)
    seq.set_guesses_for(m.fs.precipitator.cv_precipitate[0], tear_guesses2)
    seq.set_guesses_for(m.fs.leach.liquid_inlet, tear_guesses3)
    seq.set_guesses_for(m.fs.solex_rougher.mscontactor.aqueous_inlet, tear_guesses4)

    # Initialize each stream in the flowsheet
    def function(stream):
        initializer_feed = FeedInitializer()
        initializer_product = ProductInitializer()
        initializer1 = MSContactorInitializer()
        initializer2 = BlockTriangularizationInitializer()

        propagate_state(m.fs.liq_feed)
        propagate_state(m.fs.sol_feed)

        if stream == m.fs.leach_liquid_feed:
            initializer_feed.initialize(m.fs.leach_liquid_feed)
        elif stream == m.fs.leach_solid_feed:
            initializer_feed.initialize(m.fs.leach_solid_feed)
        elif stream == m.fs.leach_filter_cake:
            print(f"Initializing {stream}")
            initializer_product.initialize(m.fs.leach_filter_cake)
        elif stream == m.fs.leach:
            print(f"Initializing {stream}")
            try:
                initializer1.initialize(m.fs.leach)
            except:
                # Fix feed states
                m.fs.leach.liquid_inlet.flow_vol.fix()
                m.fs.leach.liquid_inlet.conc_mass_comp.fix()
                m.fs.leach.solid_inlet.flow_mass.fix()
                m.fs.leach.solid_inlet.mass_frac_comp.fix()
                # Re-solve leach unit
                solver = SolverFactory("ipopt")
                solver.solve(m.fs.leach, tee=True)
                # Unfix feed states
                m.fs.leach_liquid_feed.flow_vol.unfix()
                m.fs.leach.liquid_inlet.conc_mass_comp.unfix()
                m.fs.leach.solid_inlet.flow_mass.unfix()
                m.fs.leach.solid_inlet.mass_frac_comp.unfix()
        elif stream == m.fs.leach_mixer:
            initializer2.initialize(m.fs.leach_mixer)
        elif stream == m.fs.solex_rougher.mscontactor:
            print(f"Initializing {stream}")
            initializer2.initialize(m.fs.solex_rougher)
        elif stream == m.fs.solex_cleaner.mscontactor:
            print(f"Initializing {stream}")
            initializer2.initialize(m.fs.solex_cleaner)
        elif stream == m.fs.precipitator:
            print(f"Initializing {stream}")
            try:
                initializer2.initialize(m.fs.precipitator)
            except:
                # Fix feed states
                m.fs.precipitator.cv_precipitate[0].temperature.setlb(0)
                m.fs.precipitator.cv_aqueous.properties_in[0].flow_vol.fix()
                m.fs.precipitator.cv_aqueous.properties_in[0].conc_mass_comp.fix()
                # Re-solve precipitator unit
                solver = SolverFactory("ipopt")
                solver.solve(m.fs.precipitator, tee=True)
                # Unfix feed states
                m.fs.precipitator.cv_aqueous.properties_in[0].flow_vol.unfix()
                m.fs.precipitator.cv_aqueous.properties_in[0].conc_mass_comp.unfix()
        elif stream == m.fs.roaster:
            print(f"Initializing {stream}")
            try:
                initializer2.initialize(m.fs.roaster)
            except:
                m.fs.roaster.solid_in[0].temperature.setlb(0)
                m.fs.roaster.gas_out[0].temperature.setub(2000)
                m.fs.roaster.gas_in[0].temperature.setub(2000)
                m.fs.roaster.gas_in[0].pressure.setub(1e6)
                m.fs.roaster.gas_inlet.flow_mol.fix()
                m.fs.roaster.flow_mol_moist_feed.fix()
                m.fs.roaster.frac_comp_recovery.fix()
                # Re-solve roaster unit
                solver = SolverFactory("ipopt")
                solver.solve(m.fs.roaster, tee=True)
                m.fs.roaster.gas_inlet.flow_mol.unfix()
                m.fs.roaster.flow_mol_moist_feed.unfix()
                m.fs.roaster.frac_comp_recovery.unfix()
        else:
            print(f"Initializing {stream}")
            initializer2.initialize(stream)

    seq.run(m, function)

## Step 3.2: Add solver
Solve the model by running the flowsheet using the ipopt solver.

In [11]:
def solve(m):
    solver = SolverFactory("ipopt")
    results = solver.solve(m, tee=True)

## Step 3.3 Solve the system
Scale, initialize, and solve the model.

In [12]:
# Applies scaling to the model after the models are constructed and operating conditions are set
scaled_model = set_scaling(m)

# Initializes the scaled model
initialize_system(scaled_model)

# Solves the scaled model
solve(scaled_model)

Initialization Order
fs.leach_solid_feed
fs.leach
fs.sl_sep1
fs.leach_filter_cake
fs.sep1
fs.sx_mixer
fs.precipitator
fs.sl_sep2
fs.roaster
fs.recycle2_purge
Initializing fs.leach
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,1].conc_mass_comp[Sc]' to a
numeric value `-6.939701540306167e-13` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,1].conc_mol_comp[Sc]' to a
numeric value `-1.5440760305548552e-12` outside the bounds
(9.999999999999999e-16, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,1].conc_mass_comp[Y]' to a
numeric value `-6.040794230186165e-13` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,1].conc_mol_comp[Y]' to a
numeric value `-6.794582393549828e-13` outside the bounds
(9.99999

WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mol_comp[Nd]' to a
numeric value `-2.2311401873682956e-10` outside the bounds
(9.999999999999999e-16, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mol_comp[Sm]' to a
numeric value `5.907012634345073e-16` outside the bounds
(9.999999999999999e-16, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mass_comp[Nd]' to
a numeric value `-3.218243756818731e-10` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mass_comp[Nd]' to
a numeric value `-3.218243756818731e-10` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var
'fs.leach_mixer.recycle_state[0.0].conc_mass_comp

WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,1].conc_mass_comp[Sm]' to a
numeric value `-1.4842797037296561e-13` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,1].conc_mol_comp[Sm]' to a
numeric value `-9.871633379787146e-14` outside the bounds
(9.999999999999999e-16, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,2].conc_mass_comp[Y]' to a
numeric value `-2.5140953109872926e-13` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,2].conc_mol_comp[Y]' to a
numeric value `-2.8278499963597847e-18` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.leach.liquid[0.0,2].conc_mass_comp[Nd]' to a
numeric value `-1.55657

WARNING (W1002): Setting Var 'fs.sep1.purge_state[0.0].conc_mol_comp[Sm]' to a
numeric value `-1.1088954989379084e-18` outside the bounds
(9.999999999999999e-16, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mol_comp[Nd]' to a
numeric value `-5.608971952886966e-10` outside the bounds
(9.999999999999999e-16, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mass_comp[Nd]' to
a numeric value `-6.596072994582448e-10` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.sep1.mixed_state[0.0].conc_mass_comp[Nd]' to
a numeric value `-6.596072994582448e-10` outside the bounds (1e-20, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var
'fs.leach_mixer.recycle_state[0.0].conc_mass_com

## Step 3.4 Display the relevant outputs

In [13]:
# This display needs to be replaced with m.fs.roaster.report() when available
# m.fs.roaster.display()